In [1]:
import queue
import time
import os
import sys

from DataHandler.TradeLOBDataHandler import HistoricTradeLOBDataHandler
from Strategy.strategy import BuyAndHoldStrategy
from Portfolio.LogPlotPortfolio import LogPlotPortfolio
from Execution.execution import SimulatedExecutionHandler

event_queue = queue.Queue()
data_handler = HistoricTradeLOBDataHandler(event_queue, 
                                           symbol_list=['btc_usdt','eth_usdt','xrp_usdt'],
                                           exchange_list=['binance','binance','binance'], 
                                           file_dir = 'data_sample/small_sample/', 
                                           # file_dir = 'data_sample/',
                                           is_csv=False)


portfolio = LogPlotPortfolio(event_queue, data_handler)      # 组合
executor = SimulatedExecutionHandler(event_queue, data_handler)   # 回测模拟成交器；如果是实盘这里就是算法交易模块
strategy = BuyAndHoldStrategy(event_queue, data_handler, portfolio, executor)       # 策略实例。实际应用中应该有多个策略实例


cnt = 0
while True:
    # Update the bars (specific backtest code, as opposed to live trading)
    if data_handler.continue_backtest == True:
        data_handler.update_TradeLOB()
    else:
        break
    
    if cnt==1: break

    # Handle the events
    while True:
        try:
            event = event_queue.get(False)
        except queue.Empty:
            break
        else:
            if event is not None:

                if event.type == 'MARKET':
                    print('get market event', event)
                    strategy.calculate_signals(event)
                    portfolio.update_holdings_from_market()
                    executor.on_market_event(event)

                elif event.type == 'ORDER':
                    executor.on_order_event(event)
                    # cnt = 1
                    # break

                elif event.type == 'FILL':
                    print('get fill event', event)
                    #portfolio.update_positions_from_fill(event)
                    cnt = 1
                    break

backtest on:  ['btc_usdt_binance', 'eth_usdt_binance', 'xrp_usdt_binance']
/*----- start initialize the DataHandler -----*/
complete init of the data: btc_usdt_binance
complete init of the data: eth_usdt_binance
complete init of the data: xrp_usdt_binance
/*----- DataHandler initialization ends -----*/

===== processing market event in  1702745999998  =====
get new market events and push to queue
get market event {'type': 'MARKET'}

===== processing market event in  1702745999999  =====
get new market events and push to queue
get market event {'type': 'MARKET'}

===== processing market event in  1702746000000  =====
get new market events and push to queue
get market event {'type': 'MARKET'}

===== processing market event in  1702746000001  =====
get new market events and push to queue
get market event {'type': 'MARKET'}

===== processing market event in  1702746000002  =====
get new market events and push to queue
get market event {'type': 'MARKET'}

===== processing market event in  1

KeyError: 1702746000024

In [ ]:
event.timestamp

1702746000100

In [ ]:
tmp = {192:'a', 182:'e', 196:'r',200:None}
list(tmp.keys())

[192, 182, 196, 200]

In [ ]:
sorted(tmp.items(), key=lambda x:x[0])

[(182, 'e'), (192, 'a'), (196, 'r'), (200, None)]

In [ ]:
tmp[200] ==False

False

In [ ]:
tmp[182]

'e'

In [ ]:
dict( (k,v) for k, v in [(s, {}) for s in ['as','bb']] )

{'as': {}, 'bb': {}}

In [ ]:
tmp = 'xrp_usdt_binance'
tmp.split("_")[-1]

'binance'

In [ ]:
import pandas as pd

name = 'xrp_usdt_binance_trade.parquet'
tmp = pd.read_parquet('data_sample/'+name)
tmp = tmp.loc[tmp.time<1702756000000,]
tmp.to_parquet('data_sample/small_sample/'+name)